### Считывание данных

In [10]:
import yadisk # https://pypi.org/project/yadisk/
import os
import httpx
from dotenv import load_dotenv

In [2]:
load_dotenv()
TOKEN = os.getenv('DEBUG_TOKEN')

In [3]:
client = yadisk.Client(token=TOKEN)

In [15]:
with client:
    # Проверяет, валиден ли токен
    print(client.check_token())

    # Выводит содержимое "disk:/SLR Project"
    files = list(client.listdir("disk:/SLR Project"))
    print(len(files))

True
370


### Получим базовое представление о датасете

In [19]:
files[0]

<SyncResourceObject{'antivirus_status': 'clean', 'file': 'https://downloader.disk.yandex.ru/disk/9f82fac28e5389083bbe1c5e95b10239e33448b558f1cb3394a21062ecb89fef/6726b4c1/coZdVrom3SVDBmKpHCqXcKWSlinjDe-7NQmwqvnVD-ooAPxLmJoH_LdJWKBPnZgSdTfuyQNxtKdA1WdsbjY2GQ%3D%3D?uid=1627832421&filename=%23%D0%A3%D0%A1%D0%9B%D0%AB%D0%A8%D0%AC%D0%9C%D0%95%D0%9D%D0%AF%20%D0%93%D0%BB%D1%83%D1%85%D0%B0%D1%8F%20%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0.%20%D0%A1%20%D1%81%D1%83%D0%B1%D1%82%D0%B8%D1%82%D1%80%D0%B0%D0%BC%D0%B8.mp4&disposition=attachment&hash=&limit=0&content_type=video%2Fmp4&owner_uid=1627832421&fsize=44299500&hid=ea01fa8cb9fa21f738ebba19de20352b&media_type=video&tknv=v2&etag=79d41eda029f11270506034a293d7ef7', 'size': 44299500, 'public_key': None, 'sha256': '34120c9f4351a6425c75696d70f6fed793ab74719f8d5c014a9bdfa1a291e35a', 'embedded': None, 'name': '#УСЛЫШЬМЕНЯ Глухая Москва. С субтитрами.mp4', 'exif': <EXIFObject{'date_time': datetime.datetime(2023, 4, 29, 16, 18, 1, tzinfo=datetime.timezone.utc)

In [ ]:
client.get_meta(files[0]['path'])

<SyncResourceObject{'antivirus_status': 'clean', 'file': 'https://downloader.disk.yandex.ru/disk/3bea05942eb37b4a815d6f7d523a7397669b953cfbc85cf281433e7f00f57ac1/6726b710/coZdVrom3SVDBmKpHCqXcKWSlinjDe-7NQmwqvnVD-ooAPxLmJoH_LdJWKBPnZgSdTfuyQNxtKdA1WdsbjY2GQ%3D%3D?uid=1627832421&filename=%23%D0%A3%D0%A1%D0%9B%D0%AB%D0%A8%D0%AC%D0%9C%D0%95%D0%9D%D0%AF%20%D0%93%D0%BB%D1%83%D1%85%D0%B0%D1%8F%20%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0.%20%D0%A1%20%D1%81%D1%83%D0%B1%D1%82%D0%B8%D1%82%D1%80%D0%B0%D0%BC%D0%B8.mp4&disposition=attachment&hash=&limit=0&content_type=video%2Fmp4&owner_uid=1627832421&fsize=44299500&hid=ea01fa8cb9fa21f738ebba19de20352b&media_type=video&tknv=v2&etag=79d41eda029f11270506034a293d7ef7', 'size': 44299500, 'public_key': None, 'sha256': '34120c9f4351a6425c75696d70f6fed793ab74719f8d5c014a9bdfa1a291e35a', 'embedded': None, 'name': '#УСЛЫШЬМЕНЯ Глухая Москва. С субтитрами.mp4', 'exif': <EXIFObject{'date_time': datetime.datetime(2023, 4, 29, 16, 18, 1, tzinfo=datetime.timezone.utc)